In [1]:
import numpy as np
import pickle
import pandas as pd
import statistics
from statistics import mean
from scipy import stats
from biosppy.signals import ecg, eda, emg, resp
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, pearsonr
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import cmath
import matplotlib.pyplot as plt
import os
from scipy.fft import fft2, fftshift, fft
from scipy import signal
import math
import seaborn
import librosa
import pyapril
import statistics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import itertools

# Model

In [2]:
df=pd.read_pickle('D:/feat/Capt/feat_capt_main_2.pkl')
#Main_line
features=df.drop(columns=[df.columns[0], df.columns[1], df.columns[2]])
featur=np.zeros((len(df), len((df['max'][0]))*4+1))
for i in range(len(df)):
    featur[i]=np.concatenate((features['max'][i], features['min'][i], features['mean'][i], features['std'][i], features['length'][i]), axis=None)
xtrain, xtest, ytrain, ytest = train_test_split(featur,df['Activity'], test_size=0.30, random_state=101)    

In [3]:
#Logistic regression
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression(max_iter=1000000, C=1)
logmodel.fit(xtrain,ytrain)
y_pred = logmodel.predict(xtest)
y_predtrain=logmodel.predict(xtrain)
print("Train:",accuracy_score(ytrain, y_predtrain))
print("Test:",accuracy_score(ytest, y_pred))

Train: 0.9989339019189766
Test: 0.9975124378109452


# Classify as activity

In [4]:
def classify_activity(file):
    n=2
    spec = np.genfromtxt(file, delimiter=',')
    #Extract the person and activity (to test prediction)
    label = file.split('P')[0]
    person = int(file.split('P')[1].split('I')[0])
    main_line=np.zeros((spec.shape[1]))
    index=np.arange(770,830)
    
    #Main line
    for i in range(spec.shape[1]):
        spec1=np.delete(spec[:,i], index)
        if (abs(spec1.min())-abs(spec1.max())>10):
            if (np.argmax(spec1)>770):
                main_line[i]=np.argmax(spec1)+60
            else:
                main_line[i]=np.argmax(spec1)
        else:
            main_line[i]= 800
            
    #Length
    leng=int(len(main_line)/n)
    l=0
    for i in range(spec.shape[1]):
        if(abs(np.amax(spec[:,i]))>20):
            l=l+1
    ev_len=l/spec.shape[1]
    
    #Store in dataframe
    maximum=np.zeros(n)
    minimum=np.zeros(n)
    mean=np.zeros(n)
    std=np.zeros(n)
    for i in range(n):
        spec2=main_line[i*leng:(i+1)*leng]
        maximum[i]=spec2.max()
        minimum[i]=spec2.min()
        mean[i]=np.mean(spec2)
        std[i]=np.std(spec2)
    d = {'Person': person, 
         'Activity': label,
         'length': ev_len,
         'max': [maximum],
         'min': [minimum],
         'mean': [mean],
         'std': [std]
        }
    df=pd.DataFrame(data=d, index=[0])
    return df

In [5]:
file='D:\Captured_MTI/1P1I7R1_1.csv'
df1=classify_activity(file)
feat=np.concatenate((df1['max'][0], df1['min'][0], df1['mean'][0], df1['std'][0], df1['length']), axis=None)
feat=feat.reshape(1,-1)
person=df1['Person'][0]
act=logmodel.predict(feat)
print(act)

['1']


# Extract features

In [6]:
def walkingfeatures(spec, person, irr):
    main_line=np.zeros((spec.shape[1]))
    pos=np.zeros((spec.shape[1]))
    length=[]
    len_ev=0
    for i in range(spec.shape[1]):
        #LENGTH OF WALK
        down=spec[0:800,i]
        up=spec[801:1600,i]
        if abs(sum(up))>abs(sum(down)):
            pos[i]=1
        else:
            pos[i]=0
        if i>0 and pos[i]==pos[i-1]:
            len_ev=len_ev+1
        else:
            length.append(len_ev)
            len_ev=1

        #MAIN LINE 
        for j in range(spec.shape[0]//2):
            if pos[i]==1 and spec[(spec.shape[0]//2)-j][i]>-16:
                ind=(spec.shape[0]//2)-j
            if pos[i]==0 and spec[(spec.shape[0]//2)+j][i]>-16:
                ind=(spec.shape[0]//2)+j
        main_line[i]=ind

    #Length of movement
    length.append(len_ev)
    start=0
    for i in range(len(length)):
        if length[i]<300:
            start=start+length[i]
        else:
            break
    length=[s for s in length if s>300]
    av_len=np.mean(length)

    main_move=main_line[start:start+sum(length)]
    pos1=pos[start:start+sum(length)]

    maximums=[]
    minimums=[]
    #MAXS and MINS
    for i in range(15,len(main_move)):
        line=main_move[i-15:i+16]
        if line[len(line)//2+1]==line.max():
            maximums.append(i)
        if line[len(line)//2+1]==line.min():
            minimums.append(i)
    #Delete repeated values
    delete=[]
    for i in range(len(maximums)-1):
            if maximums[i]-maximums[i+1]>-10:
                delete.append(i)
    for index in sorted(delete, reverse=True):
        del maximums[index]
    delete=[]
    for i in range(len(minimums)-1):
            if minimums[i]-minimums[i+1]>-10:
                delete.append(i)
    for index in sorted(delete, reverse=True):
        del minimums[index]

    #Peak walking and leg velocity
    peak_leg_low=0
    peak_walk_low=0
    peak_walk_high=0
    peak_leg_high=0
    if 1 in pos1 and 0 in pos1:
        peak_leg_low=800-main_move[minimums].min()
        peak_walk_low=800-main_move[maximums].min()
        peak_leg_high=main_move[maximums].max()-800
        peak_walk_high=main_move[minimums].max()-800
        peak_leg=(peak_leg_low+peak_leg_high)/2
        peak_walk=(peak_walk_low+peak_walk_high)/2
    else:
        if 0 in pos1:
            peak_leg=main_move[maximums].max()-800
            peak_walk=main_move[minimums].max()-800
        else: 
            if 1 in pos1:
                peak_leg=800-main_move[minimums].min()
                peak_walk=800-main_move[maximums].min()

    #Time between steps
    dist_min=[]
    for i in range(1,len(minimums)):
        if pos[minimums[i]]==1:
            dist_min.append(minimums[i]-minimums[i-1])
    dist_max=[]
    for i in range(1, len(maximums)):
        if pos[maximums[i]]==0:
            dist_max.append(maximums[i]-maximums[i-1])
    dist=dist_max+dist_min
    dist=np.array(dist)
    mean = np.mean(dist)
    standard_deviation = np.std(dist)
    distance_from_mean = abs(dist - mean)
    max_deviations = 2
    not_outlier = distance_from_mean < max_deviations * standard_deviation
    final_dist = dist[not_outlier]
    dist_steps=np.mean(final_dist)

    #Store in dataframe
    d = {'Person': person, 
        'Irregularity' :irr,
        'Dist_steps': dist_steps,
        'Peak_walk': peak_walk,
        'Peak_legs': peak_leg,
        'Len_move': av_len
        }

    try:
        df2=pd.DataFrame(data=d, index=[0])
        df=df.append(df2, ignore_index=True)
    except: 
        df=pd.DataFrame(data=d, index=[0])
    return df

In [7]:
def sittingfeatures(spec, person, irr, act):
    main_line=np.zeros((spec.shape[1]))
    pos=np.zeros((spec.shape[1]))
    df= pd.DataFrame(columns = ['Person', 'Repetition', 'Activity','Irregularity', 'Max_velocity', 'Dropoff', 'Len_move'])
    length=[]
    len_ev=0
    for i in range(spec.shape[1]):
        #LENGTH OF WALK
        down=spec[0:800,i]
        up=spec[801:1600,i]
        if abs(sum(up))>abs(sum(down)):
            pos[i]=1
        else:
            pos[i]=0
        if i>0 and pos[i]==pos[i-1]:
            len_ev=len_ev+1
        else:
            length.append(len_ev)
            len_ev=1

        #MAIN LINE 
        for j in range(spec.shape[0]//2):
            if pos[i]==1 and spec[(spec.shape[0]//2)-j][i]>-16:
                ind=(spec.shape[0]//2)-j
            if pos[i]==0 and spec[(spec.shape[0]//2)+j][i]>-16:
                ind=(spec.shape[0]//2)+j
        main_line[i]=ind
        
    if act=='2':
        #Max value
        max_velocity=main_line.max()-800
        #Dropoff
        vel_arg=np.argmax(main_line)
        for i in range(vel_arg, len(main_line)):
            if main_line[i]<=815:
                dropoff_arg=i
                break
    else:
        #Max_value
        max_velocity=800-main_line.min()
        #Dropoff
        vel_arg=np.argmin(main_line)
        for i in range(vel_arg, len(main_line)):
            if main_line[i]>=785:
                dropoff_arg=i
                break
                
    dropoff=dropoff_arg-vel_arg
    
    #length of event
    l=0
    
    for i in range(spec.shape[1]):
        if(abs(np.amax(spec[:,i]))<20):
            l=l+1
    ev_len=l/spec.shape[1]
    
    #Store in dataframe
    d = {'Person': person, 
        'Irregularity' :irr,
        'Max_velocity': max_velocity,
        'Dropoff': dropoff,
        'Len_move': ev_len
        } 
        
    
    df=pd.DataFrame(data=d, index=[0])
    return df

In [8]:
spec = np.genfromtxt(file, delimiter=',')
irr=file.split('/')[1]
irr = irr.split('I')[1][0]
if act=='1':
    walk_feat=walkingfeatures(spec,df1['Person'], irr)
    baseline=pd.read_pickle('D:/Baseline/walk.pkl')
    walk=baseline.iloc[person-1]
else:
    if act=='2':
        sit_feat=sittingfeatures(spec,df1['Person'], irr, act)
        baseline=pd.read_pickle('D:/Baseline/sit.pkl')
        sit=baseline.iloc[person-1]
    else:
        stand_feat=sittingfeatures(spec,df1['Person'], irr, act)
        baseline=pd.read_pickle('D:/Baseline/stand.pkl')
        stand=baseline.iloc[person-1]

# Compare baseline and instance

In [13]:
if act=='1':
    #Feedback for time between steps
    if walk_feat['Dist_steps'][0]>walk['Dist_steps']+4*walk['std_steps']:
        print('The time between your steps is greater than normal')
    else:
        if walk_feat['Dist_steps'][0]<walk['Dist_steps']-4*walk['std_steps']:
            print('The time between your steps is lower than normal')
        else:
            print('The time between your steps is normal')

    #Feedback for speed of walk
    if walk_feat['Peak_walk'][0]>walk['Peak_walk']+4*walk['std_walk']:
        print('The speed of your walk is greater than normal')
    else:
        if walk_feat['Peak_walk'][0]<walk['Peak_walk']-4*walk['std_walk']:
            print('The speed of your walk is lower than normal')
        else:
            print('The speed of your walk is normal')

    #Feedback for speed of walk
    if walk_feat['Peak_legs'][0]>walk['Peak_legs']+4*walk['std_legs']:
        print('The speed of your legs is greater than normal')
    else:
        if walk_feat['Peak_legs'][0]<walk['Peak_legs']-4*walk['std_legs']:
            print('The speed of your legs is lower than normal')
        else:
            print('The speed of your legs is normal')

    #Feedback for time of movement
    if walk_feat['Len_move'][0]>walk['Len_move']+4*walk['std_len']:
        print('It took you longer than usual to walk this distance')
    else:
        if walk_feat['Len_move'][0]<walk['Len_move']-4*walk['std_len']:
            print('It took you less than usual to walk this distance')
        else:
            print('You walked this distance in a normal amount of time')
else:
    if act=='2':
        if sit_feat['Max_velocity'][0]>sit['Max_velocity']+4*sit['std_velocity']:
            print('You are descending faster than normally')
        else:
            if sit_feat['Max_velocity'][0]<sit['Max_velocity']-4*sit['std_velocity']:
                print('You are descending slower than normally')
            else:
                print('You are descending at a normal speed')

        #Feedback for speed of walk
        if sit_feat['Dropoff'][0]>sit['Dropoff']+4*sit['std_dropoff']:
            print('You let yourself fall on the chair too hard')
        else:
            if sit_feat['Dropoff'][0]<sit['Dropoff']-4*sit['std_dropoff']:
                print('You let yourself fall less than normally')
            else:
                print('You let yourself fall normally')

        #Feedback for speed of walk
        if sit_feat['Len_move'][0]>sit['Len_move']+4*sit['std_len']:
            print('You took longer than normal to sit')
        else:
            if sit_feat['Len_move'][0]<sit['Len_move']-4*sit['std_len']:
                print('You took less than normal to sit')
            else:
                print('You took a normal amount of time to sit')
    else:
        if stand_feat['Max_velocity'][0]>stand['Max_velocity']+4*stand['std_velocity']:
            print('You got up faster than normally')
        else:
            if stand_feat['Max_velocity'][0]<stand['Max_velocity']-4*stand['std_velocity']:
                print('You got up slower than normally')
            else:
                print('You got up at a normal speed')

        #Feedback for speed of walk
        if stand_feat['Dropoff'][0]>stand['Dropoff']+4*stand['std_dropoff']:
            print('You stopped moving slower than usual' )
        else:
            if stand_feat['Dropoff'][0]<stand['Dropoff']-4*stand['std_dropoff']:
                print('You stopped moving faster than usual')
            else:
                print('You stopped moving normally')

        #Feedback for speed of walk
        if stand_feat['Len_move'][0]>stand['Len_move']+4*stand['std_len']:
            print('You took longer than normal to stand up')
        else:
            if stand_feat['Len_move'][0]<stand['Len_move']-4*stand['std_len']:
                print('You took less than normal to stand up')
            else:
                print('You took a normal amount of time to stand up')

The time between your steps is normal
The speed of your walk is lower than normal
The speed of your legs is lower than normal
It took you longer than usual to walk this distance
